In [1]:
# use only the second GPU
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_VISIBLE_DEVICES=1


In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle

## Load the trained model

In [3]:
checkout_dir = 'checkpoints/'

text, words_to_ids, ids_to_words = pickle.load(open('data/preprocess.p', mode='rb'))
sess = tf.Session()

loader = tf.train.import_meta_graph(checkout_dir + 'model.meta')
loader.restore(sess, checkout_dir + 'model')

loaded_graph = tf.get_default_graph()
# Get Tensors from loaded model
names = ["input:0", "initial_state:0", "final_state:0", "probs:0"]
tensors = [loaded_graph.get_tensor_by_name(name) for name in names]
inputs, initial_state, final_state, probs = tensors

INFO:tensorflow:Restoring parameters from checkpoints/model


In [4]:
from tokenizer import *

def pick_word(probabilities, int_to_vocab):
    return int_to_vocab[np.argmax(probabilities)]

def generate(starting_text='Homer Simpson:', generate_length=300):
    sentence_tokens = text_to_tokens(starting_text)
    prev_state = sess.run(initial_state, {inputs: np.array([[1]])})

    for n in range(generate_length):
        sentence_ids = [[words_to_ids[word] for word in sentence_tokens]]
        sentence_len = len(sentence_ids[0])

        probabilities, prev_state = sess.run(
            [probs, final_state],
            {inputs: sentence_ids, initial_state: prev_state})

        predicted_token = pick_word(probabilities[0][sentence_len-1], ids_to_words)
        sentence_tokens.append(predicted_token)
    return tokens_to_text(sentence_tokens)

## Generate some text

In [5]:
generate(starting_text='Homer Simpson:')

"Homer Simpson: I don't know how to do this thing.\n\nHomer Simpson: I don't know how to do this thing to do with a  dog.\n\nHomer Simpson: I don't know how to do this thing.\n\nHomer Simpson: I don't know how to do this thing to do with a  dog.\n\nHomer Simpson: I don't know how to do this thing.\n\nHomer Simpson: I don't know how to do this thing to do with a  dog.\n\nHomer Simpson: I don't know how to do this thing.\n\nHomer Simpson: I don't know how to do this thing to do with a  dog.\n\nHomer Simpson: I don't know how to do this thing.\n\nHomer Simpson: I "